# Flash calculations

Flash, or more general phase separation calculations is the process of determining the equilibrium state of a system under conditions specified by thermodynamic state functions.
At equilibrium the available mass splits into one or multiple physical states like liquid or gas.
To specify the thermodynamic state, three state quantities are required:

- mass of present components/ chemical species (mole numbers, overall compositions, ...)
- state function related to work energy (pressure or volume)
- state function related to thermal energy (temperature, internal energy, enthalpy)

PorePy provides some functionality to determine the equilibrium state of fluid mixtures.
It does **not** cover the full range of possible thermodynamic calculations and state definitions.
Instead it focuses on the properties and state definitions relevant for thermal compositional flow in the subsurface as indicated above.
One core ability of PorePy's flash framework is the ability to solve large amounts of flash calculations very efficiently,
making it suitable for cell-wise applications in flow simulations.

This tutorial explains the flash API and presents example calculations using a compiled persistent-variable flash.
A familiarity with the [tutorial on fluid modeling](./fluid_modeling.ipynb) is of advantage.

### Table of Contents

1. [Usage of numba](#usage-of-numba)
2. [General flash API](#general-flash-api)
3. [Example: Persistent-variable flash](#example-persistent-variable-flash)
4. [Flash solvers](#flash-solvers)
5. [Example: Semi-smooth Newton solver](#example-a-semi-smooth-newton-solver)
6. [Conclusion](#conclusion)

## Usage of numba

Available flash implementions in PorePy rely heavily in [numba](https://numba.pydata.org/).<br>
What you need to know as a user is that 

1. The framework relies on just-in-time compilation.
2. Compilation can be disabled for debugging or smaller problems.

The flash framework has certain levels of abstractions which require some ahead-of-time compilation.
While this is not yet fully supported by numba, we achieve a similar effect with static signature typing, which forces compilation ahead of usage.<br>
*This leads to a certain overhead in execution time when importing the flash module for the first time, and when compiling the flash for a specific fluid.*<br>
It also requires an explicit compilation step before the flash calculations are available.

To disable compilation and just use the pure Python implementation, do the following:

In [1]:
import os
# os.environ["NUMBA_DISABLE_JIT"] = "1"

By uncommenting this flag, numba compilation is disabled and you will run the module with pure Python code.

The first thing you will notice is that it is significantly slower, but the calculations will start right away.
This is especially handy if you want to solve only a few problems to get an idea of how the mixture behaves, or you need some concrete values (exercise!).

## General flash API

The flash is essentially the solution of a challenging system of algebraic and nonlinear equations.
While the structure of the equations is always the same, individual terms depend on the present components, their physical parameters and the underlying equation of state.

Therefore, the flash is implemented as an object containing templates for those equations.
It populates them with the information provides by an instance of [Fluid](../src/porepy/compositional/base.py#706).

The generel flash is hence instatiated using some fluid object.

In [2]:
from inspect import getsource, signature

import porepy.compositional.flash as pf

print(signature(pf.AbstractFlash.__init__))

(self, fluid: 'pp.Fluid[pp.FluidComponent, pp.Phase]', params: 'Optional[dict]' = None) -> 'None'


Once the flash object is created, the interface for the flash is as follows:

In [3]:
print(getsource(pf.AbstractFlash.flash))

    @abc.abstractmethod
    def flash(
        self,
        specification: StateSpecType,
        z: Optional[Sequence[np.ndarray | pp.number]] = None,
        /,
        *,
        initial_state: Optional[pp.compositional.FluidProperties] = None,
        params: Optional[dict] = None,
        **kwargs,
    ) -> FlashResults:
        """Abstract method for performing a flash procedure.

        The equilibrium state must be defined in terms of compositions ``z`` and two
        state functions declared in ``specification``.
        One state must relate to pressure or volume. The other to energy.

        Parameters:
            specifications: Equilibrium specifications in terms of state functions.
            z: ``default=None``

                Overall fractions of mass per component.

                It is only optional for pure fluids (``z`` is implicitly assumed to be
                1). For fluid mixtures with multiple components it must be of length
                ``num_compo

The minimum we need are equilibrium `specifiations` in terms of two state functions.
PorePy supports two types of state specifiations:

- Isobaric specifiations (pressure at equilibrium is given)
- Isochoric specifiations (specifiv volume of the fluid at equilibrium is given)

In [4]:
print(getsource(pf.IsobaricSpecifications))

class IsobaricSpecifications(TypedDict):
    """Typed dictionary for isobaric equilibrium specifications.

    The pressure values are obligatory and one energy-related variable is required.

    """

    p: np.ndarray | pp.number
    """Pressure at equilibrium."""

    T: NotRequired[np.ndarray | pp.number]
    """Temperature at equilibrium."""

    h: NotRequired[np.ndarray | pp.number]
    """Specific fluid enthalpy at equilibrium."""



We see that isobaric specifiations have two additional optional fields, temperature and specific fluid enthalpy.
One of them must be given additionally.
Providing pressure and enthalpy values invokes the ph-flash for example.

Additionally to the state functions, the mass must be provided in form of overall composisitions `z` (fractions of total mass associated with a component in the fluid). <br>
If the fluid has only 1 component, this argument is not required and implicitly asssumed to be one.

The `initial_state` argument can be used to provide an initial guess for the flash problem. Otherwise the flash object has to compute them. *Note that this is not a trivial task in thermodynamics and can contribute noticable to the overall time spent to solve the problem*.

The return value is a [FlashResults](../src/porepy/compositional/flash/abstract_flash.py##116) data structure, which is an extension of [FluidProperties](../src/porepy/compositional/states.py#231).
Additionally to all fluid properties evaluated at equilibrium, it also contains some metrics of the performed flash, like convergence flags and number of iterations.
This structure can be used to feed values into a global flow system for example.

## Example: Persistent-variable flash

## Flash solvers

## Example: A semi-smooth Newton solver

## Conclusion